In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
#test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
#df_test = test['comment_text']
df = train[['target','comment_text']]

In [3]:
from imblearn.under_sampling import RandomUnderSampler
X = df['comment_text']
X = X.values.reshape(-1,1)
y = df['target']
y = np.where(y>=0.5,1.,0.)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_sample, y_sample = undersample.fit_resample(X, y)

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_sample.reshape(-1), y_sample, 
                                                    test_size=0.3, random_state=42)

In [5]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
vocab_sz = 10000
maxlen=311
tok = Tokenizer(num_words=vocab_sz, oov_token='UNK')
tok.fit_on_texts(X_sample.reshape(-1))
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test )
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [6]:
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, GRU, Dropout,Embedding ,Dense
model = Sequential()
model.add(Embedding(vocab_sz+1, 50, input_length=maxlen))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.summary()

2022-06-02 19:46:06.226744: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 311, 50)           500050    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 529,555
Trainable params: 529,555
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=64, epochs=3,validation_data=(x_test, y_test))

2022-06-02 19:46:06.871032: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
3158/3158 [==============================] - 1100s 348ms/step - loss: 0.3530 - accuracy: 0.8484 - val_loss: 0.3010 - val_accuracy: 0.8759
Epoch 2/3
3158/3158 [==============================] - 1110s 352ms/step - loss: 0.2876 - accuracy: 0.8812 - val_loss: 0.2943 - val_accuracy: 0.8787
Epoch 3/3
3158/3158 [==============================] - 1076s 341ms/step - loss: 0.2694 - accuracy: 0.8881 - val_loss: 0.2964 - val_accuracy: 0.8781


In [8]:
model.evaluate(x_test, y_test)

2707/2707 [==============================] - 197s 73ms/step - loss: 0.2964 - accuracy: 0.8781


[0.2963508069515228, 0.878119170665741]

In [9]:
test_df= pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
test_df= test_df['comment_text']
test_df.head()

0    [ Integrity means that you pay your debts.]\n\...
1    This is malfeasance by the Administrator and t...
2    @Rmiller101 - Spoken like a true elitist. But ...
3    Paul: Thank you for your kind words.  I do, in...
4    Sorry you missed high school. Eisenhower sent ...
Name: comment_text, dtype: object

In [10]:
xt = tok.texts_to_sequences(test_df)
xt = pad_sequences(xt, maxlen=maxlen)

In [11]:
y_pred= model.predict(xt)
y_pred = np.where(y_pred>=0.5,1.,0.)
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = y_pred
sub_df.to_csv('submission.csv',index = False)